In [1]:
import neo4j
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_neo4j import init, geofoxpois_insert_query
from geofox_client import get_geofox_client
from utils_geofox import get_pois, poisdf2rows
import pandas as pd

c:\Users\paul-\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [3]:
client = get_geofox_client()

In [4]:
# Fetch the POIs
pois = get_pois(None, maxList=10000, loadFromDisk=True)

In [5]:
len(pois['results'])

4917

In [6]:
pois['results'][0]

{'name': 'Amt für Immissionsschutz und Betriebe',
 'city': 'Hamburg',
 'combinedName': 'Amt für Immissionsschutz und Betriebe',
 'id': '0',
 'type': 'POI',
 'coordinate': {'x': 10.004024, 'y': 53.497556},
 'address': 'Neuenfelder Straße 19 (Hamburg)'}

In [7]:
df_pois = pd.DataFrame(pois['results'])

In [8]:
df_pois.columns.to_list()

['name', 'city', 'combinedName', 'id', 'type', 'coordinate', 'address']

In [9]:
impcols = ['name', 'city', 'id', 'coordinate', 'address']

In [10]:
df_pois_impcols = df_pois[impcols]

In [11]:
df_pois_impcols.coordinate.isna().sum()

np.int64(0)

In [12]:
rows = poisdf2rows(df_pois_impcols)

In [13]:
rows

[{'geofoxid': '0',
  'name': 'Amt für Immissionsschutz und Betriebe',
  'city': 'Hamburg',
  'address': 'Neuenfelder Straße 19 (Hamburg)',
  'longitude': 10.004024,
  'latitude': 53.497556},
 {'geofoxid': '1',
  'name': 'Behörde für Stadtentwicklung und Wohnen',
  'city': 'Hamburg',
  'address': 'Neuenfelder Straße 19 (Hamburg)',
  'longitude': 10.004187,
  'latitude': 53.497465},
 {'geofoxid': '2',
  'name': 'Behörde für Umwelt, Klima Energie und Agrarwirtschaft',
  'city': 'Hamburg',
  'address': 'Neuenfelder Straße 19 (Hamburg)',
  'longitude': 10.004217,
  'latitude': 53.497465},
 {'geofoxid': '3',
  'name': 'Behörde für Wirtschaft und Innovation',
  'city': 'Hamburg',
  'address': 'Alter Steinweg 4 (Hamburg)',
  'longitude': 9.983989,
  'latitude': 53.550549},
 {'geofoxid': '4',
  'name': 'Referat Außenwirtschaft',
  'city': 'Hamburg',
  'address': 'Alter Steinweg 4 (Hamburg)',
  'longitude': 9.983685,
  'latitude': 53.550434},
 {'geofoxid': '1000',
  'name': 'Agentur für Arbeit A

In [ ]:
# Execute the query
with driver.session() as session:
    result = session.run(geofoxpois_insert_query, rows=rows)
    print(f"Imported {result.single()['total']} pois")

driver.close()

Imported 4917 stations
